## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [1]:
import pickle

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

In [2]:
set_ = (1, 2, 3, 4)
set_ = set(set_)
len(set_)

4

Всего в корпусе 1384 пары вопрос-ответ

In [3]:
len(qa_corpus)

1384

Первый элемент блока это вопрос, второй - ответ на него

In [4]:
qa_corpus[0][1]

'Добрый вечер!Из Вашего вопроса вообще ничего не ясно.Ваш сын по ВНЖ в Нижегородской обл. сделал временную\xa0 на 90 дней в Ростове? Так? Или в чем заключается вопрос?С ув., АлёнаМиграционный юристРостов-на-Дону '

In [5]:
questions = []
answers = []
for corp in qa_corpus:
    questions.append(corp[0])
    answers.append(corp[1])

In [6]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models import Word2Vec, KeyedVectors
from stop_words import get_stop_words
import string
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
mystem = Mystem()
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import json
import numpy as np

In [7]:
model_path = 'D:/tayga_1_2.vec'
model = KeyedVectors.load_word2vec_format(model_path, binary=False)

In [8]:
from gensim import matutils


def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

In [27]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(get_stop_words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]
#     lemmas = morph.parse(x)[0].normal_form

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        pos = morph.parse(lemma)[0].tag.POS
        lemma = lemma + '_' + str(pos)
        lemmas_arr.append(lemma)
#     lemmas_ = ' '.join(lemmas_arr)
    return lemmas_arr

def get_w2v_vectors(lemmas_arr, text):
    """Получает вектор документа"""
    dict_ = {}
    feature_vec = np.zeros((300, ), dtype='float32')
    
    for lemma in lemmas_arr:
        try:
#             my_vec.append(model.wv[lemma].tolist())
            feature_vec = np.add(feature_vec, model.wv[lemma])
#             print(len(model.wv[lemma]))
        except KeyError:
            print(lemma)
            feature_vec = np.add(feature_vec, np.zeros((300, ), dtype='float32'))
    text = text.strip()
    dict_ = {text :  feature_vec}
#     print(dict_)
#     print(dict_)
#     json_ = json.dumps(dict_, ensure_ascii=False)
    return dict_
  

def save_w2v_base(dict_, base_dict):
    """Индексирует всю базу для поиска через word2vec"""

    base_dict.update(dict_)
#     print(base_dict)
#     base_json = json.dumps(base_dict, ensure_ascii=False)
    return base_dict

In [ ]:
base_dict = {}
for answer in answers:
    prepr = preprocessing(answer, del_stopwords=True, del_digit=True)
    get_wv = get_w2v_vectors(prepr, answer)
    save_wv = save_w2v_base(get_wv, base_dict)

In [29]:
import random
def w2v_search(question, base_dict):
    scores = []
#     question  = random.choice(questions)
    print('question - ', question)
    prepr = preprocessing(question, del_stopwords=True, del_digit=True)
#     print(prepr)
    get_wv = get_w2v_vectors(prepr, question)
#     print('aaa',get_wv.get(question))
#     print(type(val))
    for key, value in base_dict.items():
#         print(value)
        for key1, value1 in get_wv.items():
            
#         print(get_wv[key])
            try:
                sim = similarity(value, value1)
                scores.append((key, sim))
            except ValueError:
                print('string')
#     scores = sorted(scores, key=lambda x: x[1], reverse=True)
#     return scores[:5]
    return scores
    

# search = w2v_search(questions, save_wv)
# search

Doc2Vec

In [13]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
model = Doc2Vec.load('D:/Doc2Vec_100s_1000e')

In [25]:
def preprocessing(input_text, del_digit=True):
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]
    lemmas_arr = []
    for lemma in lemmas:
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
#     lemmas_ = ' '.join(lemmas_arr)
    return lemmas_arr


def get_d2v_vectors(lemmas_arr, text):
    """Получает вектор документа"""
    new_vec = model.infer_vector(lemmas_arr)
    text = text.strip()
    text = text.replace('\xa0', '')
    text = text.replace('\u200b', '')
    dict_ = {text :  new_vec}
    return dict_


def save_d2v_base(dict_, base_dict):
    """Индексирует всю базу для поиска через doc2vec"""
    base_dict.update(dict_)
    return base_dict

In [ ]:
base_dict = {}
for answer in answers:
    prepr = preprocessing(answer, del_digit=True)
    print(1)
    get_dv = get_d2v_vectors(prepr, answer)
    print(2)
    save_dv = save_d2v_base(get_dv, base_dict)
    print(3)

In [21]:
def d2v_search(question, save_dv):
    scores = []
#     question = random.choice(questions)
    print('question - ', question)
    prepr = preprocessing(question, del_digit=True)
    print('3')
    get_dv = get_d2v_vectors(prepr, question)
    for key, value in save_dv.items():
        for key1, value1 in get_dv.items():
            try:
                sim = similarity(value, value1)
                scores.append((key, sim))
            except ValueError:
                print('string')
#     scores = sorted(scores, key=lambda x: x[1], reverse=True)
#     return scores[:5]
    return scores
    



In [20]:
import random
search = d2v_search(questions, save_dv)
search

question -  
Я живу на третьем этаже, на первом этаже под моей квартирой находится небольшой продуктовый магазин, на пятом этаже живет пенсионерка, у которой на балконе сделана самодельная крыша со скатом. На этой крыше «выросла» огромная сосулька. Старшая по дому решила эту сосульку убрать, чтобы она случайно не упала на голову покупателям магазина. Она пригласила жильца из другого подъезда, чтобы он сбил сосульку. На моем балконе есть бельевые веревки на улице (сам балкон застеклен). Эти люди встретились мне в подъезде, когда шли наверх. Пошутили «ни о чем» про сосульку не сказали. Я ушла из дома, а когда вечером вернулась, обнаружила, что трубы с веревками почти полностью согнуты. Закреплены они под обшивкой балкона, просто так к ним не подберешься. Кто теперь должен мне это починить? Я хочу написать на них заявление, но куда? У нас управляющая компания.

3


c:\users\maria tsareva\appdata\local\programs\python\python36-32\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Я живу на третьем этаже, на первом этаже под моей квартирой находится небольшой продуктовый магазин, на пятом этаже живет пенсионерка, у которой на балконе сделана самодельная крыша со скатом. На этой крыше «выросла» огромная сосулька. Старшая по дому решила эту сосульку убрать, чтобы она случайно не упала на голову покупателям магазина. Она пригласила жильца из другого подъезда, чтобы он сбил сосульку. На моем балконе есть бельевые веревки на улице (сам балкон застеклен). Эти люди встретились мне в подъезде, когда шли наверх. Пошутили «ни о чем» про сосульку не сказали. Я ушла из дома, а когда вечером вернулась, обнаружила, что трубы с веревками почти полностью согнуты. Закреплены они под обшивкой балкона, просто так к ним не подберешься. Кто теперь должен мне это починить? Я хочу написать на них заявление, но куда? У нас управляющая компания. Здравствуйте, Ольга! Для начала напишите претензию управляющей компании, сделайте приписку, что в случае неудовлетворения Вашей претензии ос

OKAPI BM25

In [24]:
from math import log
from collections import Counter

In [25]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    russian_stopwords = set(get_stop_words('russian'))
    words = [x.lower().strip(string.punctuation + '»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]
    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [26]:
answers_ = []
for answer in answers:
    prepr = preprocessing(answer, del_stopwords=True, del_digit=True)
    answers_.append(prepr)

In [27]:
sum_len = 0
for answer in answers:
    sum_len += len(answer)
av_len = float(len(answers) / sum_len)
print(av_len)

0.001046687242440445


In [49]:
def freq(question, answers_, answers):
    question = preprocessing(question, del_stopwords=True, del_digit=True)
    counts = {}
    for answer_, answer in zip(answers_, answers):
        answer = answer.replace('\xa0', '')
        answer = answer.replace('\u200b', '')
        # text_ = preprocessing(text, del_stopwords=True, del_digit=True)
        text_len = len(answers_)
        for ques in question:
            count = Counter(answer_)[ques]
            counts[answer] = [count, text_len]
    # n = 0
    # for key, value in counts.items():
    #     dl = value[1]
    #     qf = value[0]
#     print(counts)
    return counts

In [44]:
k1 = 2.0
b = 0.75


def score_BM25(qf, dl, avgdl, k1, b, N, n, counts) -> float:
    score = log((N - n + 0.5) / (n + 0.5)) * (k1 + 1) * qf / (qf + k1 * (1 - b + b * dl / avgdl))
    return score

In [60]:
def fin_ok(question, answers, N, av_len):
    q = freq(question, answers_, answers)
    n = 0
    for key, value in q.items():
        # print(key, value)
        if value[0] != 0:
            n += 1
        dl = value[1]
        qf = value[0]
        rev_score = score_BM25(qf, dl, av_len, k1, b, N, n, q)
#         print(rev_score)
        value.append(rev_score)
#         print(value)
        value.remove(value[0])
        value.remove(value[0])
#         print(value)
    scores = sorted(q, key=q.get, reverse=True)
    return scores[:5]

In [63]:
import random
question  = random.choice(questions)
print('question - ', question)
N = len(answers)
fin = fin_ok(question, answers, N, av_len)
fin

question -  
здравствуйте! у нас вот такая ситуация:  10 января у нас заканчивается срок временного убежища, для подачи на гражданства РФ нужно чтоб этому документу было 6 месяцев. третий раз нам его продлить не захотели, потому что есть РВП, сказали забирайте укр паспорта и ставьте РВП туда и делайте перевод... у мужа просрочен паспорт, вклеена фото в 25 лет, сейчас ему 27, получается просрочка 2 года. Рвп ему в паспорт не переставили, потому что нет документа удостоверяющего личность, сказали ехать в посольство делать справку о выезде на границу и продлевать паспорт там, на укр сторону ему ехать нельзя.  у меня укр паспорт действителен до мая
, мне в мае 25 лет, РВП стоит в укр паспорте. как быть??? соотечественники есть до 8.08.2018г.... РВП и у меня и у мужа до 2019 года, но у мужа оно не где не проставлено .... можно ли заново подать документы на временное убежище??



['Добрый день! Право на постоянное проживание на территории РФ иностранным гражданам и лицам без гражданства даёт Вид на жительство (ВНЖ). В общем порядке нужно сначала получить РВП, прожить в РФ по РВП не менее года и только затем можно обратиться за получением ВНЖ. Вид на жительство выдается иностранному гражданину на пять лет и может быть продлён неограниченное количество раз. Но существует упрощённый режим получения ВНЖ, без необходимости получения РВП. Итак, получить ВНЖ, упрощённо, могут: 1) граждане р. Беларусь на основании двустороннего международного соглашения между РФ и р. Беларусь от 24.01.2006 г. 2) лица, признаные в установленном порядке носителями русского языка. Особенность данного основания заключается в следующем: а) срок действия ВНЖ составляет 3 года ( в обычном пордке — это 5 лет);  б) заявитель имеет право обращаться с заявлением о выдаче вида на жительство только в тот территориальный орган по делам миграции МВД РФ, решением комиссии которого он был признан носит